In [ ]:
# Enable autoreload
%load_ext autoreload
%autoreload 2

# Pylint parameters
%config Completer.use_jedi = False

# Measure Runtime
# !pip install ipython-autotime
# %load_ext autotime

# Mute warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
!pwd

In [ ]:
train_data = pd.read_csv('../raw_data/shopee-product-matching/train.csv') 
test_data = pd.read_csv('../raw_data/shopee-product-matching/test.csv')
ss_data = pd.read_csv('../raw_data/shopee-product-matching/sample_submission.csv', index_col = 0)

In [ ]:
train_df = train_data.copy()
test_df = train_data.copy()
ss_df = train_data.copy()

In [ ]:
train_df.size

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df['image'].shape

In [ ]:
train_df.nunique()

In [ ]:
train_df.isna().sum()

# Image preprocessing

## Resizing

We want to reshape the images in smaller size (state-of-the-art networks are trained on (224,224,3) images / tensors)

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
#import tensorflow_datasets as tfds
import pathlib

In [ ]:
batch_size = 32
img_height = 100
img_width = 100

In [ ]:
dataset_url = "../raw_data/shopee-product-matching/train_images/resized/"


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../raw_data/shopee-product-matching/train_images/',
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
os.path.dirname(os.path.abspath(os.getcwd()))

In [ ]:
    parent_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    data = 'raw_data/shopee-product-matching/train_images/resized'

In [ ]:
for value in train_data.label_group.values:
    directory = str(value)
    parent_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    data = 'raw_data/shopee-product-matching/train_images/'
    path_to_create = os.path.join(parent_dir, data, directory)
    try:
        os.mkdir(path_to_create)
        print("Directory '% s' created" % directory)
    except:
        print("File existing")
    #print(path_to_create)

In [ ]:
parent_path = os.path.join(parent_dir, data)

In [ ]:
parent_path = os.path.join(parent_dir, data)
list_dir = os.listdir(parent_path)
list_image = []
for i in os.listdir(parent_path):
    #print(i)
    if os.path.isfile(os.path.join(os.path.join(parent_dir, data),i)):
         list_image.append(i)
list_image

In [ ]:
train_data

In [ ]:
import shutil

for image in list_image:
    try:
        source = parent_path
        destination = str(list(train_data[train_data['image'] == image]['label_group'])[0])
        print(image)
        print(destination)
        print(os.path.join(source,destination))
        new_path = shutil.move(f"{source}/{image}", f"{source}/{destination}/{image}")
    except:
        print('nothing')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

!pip install cv2

# import cv2
import os

import numpy as np

In [ ]:
data_dir = pathlib.Path(parent_path)
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
check_1 = list(data_dir.glob('1429052699/*'))
PIL.Image.open(str(check_1[1]))

In [ ]:
batch_size = 64
img_height = 256
img_width = 256

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  # validation_split=0.2,
  # subset="training",
  # seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = 11015

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(64, 3, activation='tanh'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='tanh'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='tanh'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(64, activation='softmax'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=128
)

In [ ]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(64, 3, activation='tanh'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='tanh'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(32, activation='softmax'),
  layers.Dense(num_classes)
])

In [ ]:
model_2.compile(
  optimizer='RMSprop',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=128
)